---
<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 555 BioAlgorithms - Spring 2019</h1>
<h1 style="font-size: 250%;">Problem Set #4, Version 2.0</h1>
<h1>Issued Tuesday 4/9/2019; Due Tuesday, 4/23/2019</h1>
</div>

**Homework Information:** Some of the problems are probably too long to be done the night before the due date, so plan accordingly. Late problem sets will be penalized by a factor of	70.71% for each class meeting after the due date. Feel free to get help from others, but the work you submit in should be your own.

In [1]:
# Replace the following string values with the requested information
class Student:
    first = "Benjamin"
    last = "Fawcett"
    onyen = "bfawcett"
    pid = "730095208"

In [2]:
import gzip
import itertools
import math
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

def loadFasta(filename):
    """ Parses a classically formatted and possibly 
        compressed FASTA file into two lists. One of 
        headers and a second list of sequences.
        The ith index of each list correspond."""
    if (filename.endswith(".gz")):
        fp = gzip.open(filename, 'r')
    else:
        fp = open(filename, 'r')
    # split at headers
    data = fp.read().split('\n')
    fp.close()
    # ignore whatever appears before the 1st header
    sequences = []
    for sequence in data:
        # add an extra "+" to make string "1-referenced"
        sequences.append(''.join(sequence))
    return sequences

---
**Problem #1:** Of the 10 sequences provided [*here*](https://csbio.unc.edu/mcmillan/Comp555S19/data/PS4.fa), find the 3 sequences that maximize the sum of their Pairwise Global Alignments. Assume a weight of 1 for all *matches* and -1 for all *mismatches and indels* in your alignment. (Note: you only need to find the best score here, not the alignment, thus you do not need the backtracking matrix).

Report the *index* of the 3 sequences and the three alignment scores between them as a comma separated list with three rows of three numbers in the following format:

        seqIndexA, seqIndexB, AB_AlignmentScore
        seqIndexA, seqIndexC, AC_AlignmentScore
        seqIndexB, seqIndexC, BC_AlignmentScore

In [3]:
sequence = loadFasta("/Users/bbfawcett13/Documents/UNC-CH/S 2019/COMP 555/Homework/4/PS4.fa")

In [16]:
import numpy

def GlobalAlign(v, w, scorematrix, indel):
    s = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in range(0,len(v)+1):
        for j in range(0,len(w)+1):
            if (j == 0):
                if (i > 0):
                    s[i,j] = s[i-1,j] + indel
                continue
            if (i == 0):
                s[i,j] = s[i,j-1] + indel
                continue
            score = s[i-1,j-1] + scorematrix[v[i-1],w[j-1]]
            vskip = s[i-1,j] + indel
            wskip = s[i,j-1] + indel
            s[i,j] = max(vskip, wskip, score)
    return s

match = {('A','A'):  1, ('A','C'): -1, ('A','G'): -1, ('A','T'): -1,
         ('C','A'): -1, ('C','C'):  1, ('C','G'): -1, ('C','T'): -1,
         ('G','A'): -1, ('G','C'): -1, ('G','G'):  1, ('G','T'): -1,
         ('T','A'): -1, ('T','C'): -1, ('T','G'): -1, ('T','T'):  1}

v = "TTCCGAGCGTTT"
w = "TTCCGAGCGTTT"

s = GlobalAlign(v,w,match,-1)
print("Best score =", s[-1,-1])

Best score = 12


In [ ]:
v = "ATGTCAGAAATCATGAGCAATACAGCCAGGGTCAAC"
w = "CATCAGAAATCAGAGCAAGACAAATACAGCCAGGGC"

In [28]:
print(len(sequence[0]))

7477


In [27]:
%time s = GlobalAlign(sequence[0][0:2000], sequence[1][0:2000], match, -1)

CPU times: user 28.2 s, sys: 22 ms, total: 28.3 s
Wall time: 28.3 s


In [29]:
alignmentDict = {}
for i in range(0,len(sequence)):
    for j in range(i+1, len(sequence)):
        print(i,j)
        s = GlobalAlign(sequence[i], sequence[j], match, -1)
        alignmentDict[(i, j)] = s[-1,-1]

0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
2 3
2 4
2 5
2 6
2 7
2 8
2 9
3 4
3 5
3 6
3 7
3 8
3 9
4 5
4 6
4 7
4 8
4 9
5 6
5 7
5 8
5 9
6 7
6 8
6 9
7 8
7 9
8 9


In [37]:
indices = range(0, len(sequence))
x = [index for index in itertools.combinations(indices, 3)]
maxSum = 0
bestIndices = ()
for i in x:
    sumI = 0
    sumI += alignmentDict.get((i[0], i[1])) + alignmentDict.get((i[0], i[2])) + alignmentDict.get((i[1], i[2]))
    if (sumI > maxSum):
        maxSum = sumI
        bestIndices = i
        
print((bestIndices[0], bestIndices[1]), alignmentDict.get((bestIndices[0], bestIndices[1]))) 
print((bestIndices[0], bestIndices[2]), alignmentDict.get((bestIndices[0], bestIndices[2])))
print((bestIndices[1], bestIndices[2]), alignmentDict.get((bestIndices[1], bestIndices[2])))

(0, 1) 7463
(0, 2) 7443
(1, 2) 7433


In [35]:
print(alignmentDict)

{(0, 1): 7463, (0, 2): 7443, (0, 3): 7009, (0, 4): 6872, (0, 5): 6849, (0, 6): 7177, (0, 7): 7097, (0, 8): 7107, (0, 9): 7337, (1, 2): 7433, (1, 3): 6995, (1, 4): 6858, (1, 5): 6835, (1, 6): 7163, (1, 7): 7083, (1, 8): 7093, (1, 9): 7327, (2, 3): 6994, (2, 4): 6865, (2, 5): 6843, (2, 6): 7159, (2, 7): 7092, (2, 8): 7106, (2, 9): 7314, (3, 4): 6789, (3, 5): 6843, (3, 6): 6971, (3, 7): 7009, (3, 8): 7024, (3, 9): 6951, (4, 5): 6740, (4, 6): 6854, (4, 7): 7078, (4, 8): 7106, (4, 9): 6832, (5, 6): 6832, (5, 7): 6949, (5, 8): 6958, (5, 9): 6802, (6, 7): 7078, (6, 8): 7080, (6, 9): 7110, (7, 8): 7296, (7, 9): 7045, (8, 9): 7050}


---
**Problem #2:** The sequences from problem one are from endogenous retroviruses (ERVs). ERVs have a specific genomic structure, where they begin and end with a common highly conserved long terminal repeat (LTR) sequence. Using the 3 sequences you found in Problem #1, attempt to find the LTR sequences by performing a Local Alignment of the first and last 1000 bases from the same sequence. Repeat this for all 3 sequences using the same scoring matrix as in part one. Report the respective start and end indices from each of the three Local Alignments. Here you will need the backtracking matrix. Your answer should be 3 rows of 5 comma separated numbers of the form:

       seqIndexA, startOfProximalLTR, endOfProximalLTR, startOfDistalLTR, endOfDistalLTR
       seqIndexB, startOfProximalLTR, endOfProximalLTR, startOfDistalLTR, endOfDistalLTR
       seqIndexC, startOfProximalLTR, endOfProximalLTR, startOfDistalLTR, endOfDistalLTR

In [38]:
import numpy

def LocalAlign(v, w, scorematrix, indel):
    s = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    b = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in range(1,len(v)+1):
        for j in range(1,len(w)+1):
            if (j == 0):
                if (i > 0):
                    s[i,j] = max(s[i-1,j] + indel, 0)
                    b[i,j] = 1
                continue
            if (i == 0):
                s[i,j] = max(s[i,j-1] + indel, 0)
                b[i,j] = 2
                continue
            score = s[i-1,j-1] + scorematrix[v[i-1],w[j-1]]
            vskip = s[i-1,j] + indel
            wskip = s[i,j-1] + indel
            s[i,j] = max(vskip, wskip, score, 0)
            if (s[i,j] == vskip):
                b[i,j] = 1
            elif (s[i,j] == wskip):
                b[i,j] = 2
            elif (s[i,j] == score):
                b[i,j] = 3
            else:
                b[i,j] = 0
    return (s, b)

In [47]:
backDict = {}
scoreDict = {}
for i in bestIndices:
    print(i)
    s, b = LocalAlign(sequence[i][:1000], sequence[i][-1000:], match, -1)
    backDict[i] = b
    scoreDict[i] = s



0
1
2


In [78]:
def Alignment(b,v,w,i,j):
    if ((i == 0) or (j == 0)):
        return ['','']
    if (b[i,j] == 3):
        result = Alignment(b,v,w,i-1,j-1)
        result[0] += v[i-1]
        result[1] += w[j-1]
        return result
    if (b[i,j] == 2):
        result = Alignment(b,v,w,i,j-1)
        result[0] += "_"
        result[1] += w[j-1]
        return result
    if (b[i,j] == 1):
        result = Alignment(b,v,w,i-1,j)
        result[0] += v[i-1]
        result[1] += "_"
        return result

In [100]:
for i in bestIndices:
    b = backDict.get(i)
    
    result = numpy.where(s == numpy.amax(s))
    coords = list(zip(result[0], result[1]))
    v = sequence[i][:1000]
    w = sequence[i][-1000:]
    result = Alignment(b, v, w, coords[0][0], coords[0][1])
    print(len(result[0]))
    print(len(sequence[i][7476]))
    print(sequence[i][:1000].find(result[0]))
    print((len(sequence[i]) - 1000) + sequence[i][-1000:].find(result[0]))
    print(len(result[0]))

319
1
0
7158
319
319
1
0
7158
319
319
1
0
7160
319


In [4]:
print(sequence[0][0:319] == sequence[0][7158:7477])
print(sequence[1][0:319] == sequence[1][7158:7477])
print(sequence[2][0:319] == sequence[2][7160:7479])

True
True
True


---
**Problem #3:** Using the three LTRs alignments that you found in Problem #2, report a *consensus LTR sequence*.

In [87]:
seq = ""
for i in range(0, 319):
    nucleotideDict = {}
    a = sequence[0][:1000][i]
    b = sequence[1][:1000][i]
    c = sequence[2][:1000][i]
    
    x = sequence[0][-1000:][i+681]
    y = sequence[1][-1000:][i+681]
    z = sequence[2][-1000:][i+681]
    
    nucleotideDict[a] = nucleotideDict.get(a, 0) + 1
    nucleotideDict[b] = nucleotideDict.get(b, 0) + 1
    nucleotideDict[c] = nucleotideDict.get(c, 0) + 1
    nucleotideDict[x] = nucleotideDict.get(x, 0) + 1
    nucleotideDict[y] = nucleotideDict.get(y, 0) + 1
    nucleotideDict[z] = nucleotideDict.get(z, 0) + 1
    
    addNuc = ""
    val = 0
    for nuc in nucleotideDict.keys():
        if nucleotideDict.get(nuc) > val:
            val = nucleotideDict.get(nuc)
            addNuc = nuc
    
    seq = seq + addNuc
    
    
print(seq)

TGTTGCGGCCGCCAGCAGCTCGCAACGTGAACGGTTCGACTGAGAAGGCCGCTCGAGCTGTAAGAGAGGAATCTAGACGGGGCAAAAGAAGAAACGGAGCTAAGGCAATTTCATACTGATCAAAGCTCAAATTTTATTGTTGCGACACCAGTTATGAAGGAAGGGGGAGGGGACCCGATTCCCGCCGAATAATCTCTGGTCCAGTAGAAAGGTGCACGGGTGTGGCTCCGCAGGTTCCAGCAGTGGGCGTGGCAGAACGAATGAGCAGGAAGCTCCACCCCGAGCAAGCAGGTTTCAGGCTGGGGGAGGGGAGACTACA


---
**Problem #4:** Using the consensus subsequence you found in Problem #3, find a *Local Pairwise Alignment* of this consensus LTR to each of the first, and last, 1000 base pairs of the 10 original sequences and report the score for each. This should be a comma separated list of ten rows each with three numbers the following format:

         1, ProximalLTRscore, DistalLTRscore
         2, ProximalLTRscore, DistalLTRscore
                      .
                      .
                      .
         9, ProximalLTRscore, DistalLTRscore
         10, ProximalLTRscore, DistalLTRscore


In [88]:
for i in range(0, len(sequence)):
    prox = sequence[i][:1000]
    dist = sequence[i][-1000:]
    
    sp, bp = LocalAlign(prox, seq, match, -1)
    sd, bd = LocalAlign(dist, seq, match, -1)
    
    pScore = numpy.amax(sp)
    dScore = numpy.amax(sd)
    
    print(i, pScore, dScore)

0 319 319
1 315 315
2 317 317
3 294 294
4 293 291
5 282 284
6 301 299
7 293 295
8 291 293
9 319 317


---

## Instructions for submitting your problem set

When you are ready to submit a version of your problem set, follow the instructions below.

1. Press [Save and Checkpoint] on the *File* menu of your Jupyter notebook.
2. Press the link below, which will take you to a website for submitting your problem set.
3. Choose the ***correct problem set number*** from the pull-down, else you might overwrite an earlier submission.
4. Enter in your onyen and PID in the form provided, then upload your submission.

Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set

**Instructions for resubmissions:**

1. You may resubmit as many times as you like before the deadline. 
2. Resubmissions *always* overwrite any earlier submissions. 
3. If you resubmit after the due date, you will be warned of any penalties. 
4. Problem sets will not be regraded.